In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
from neuralprophet import save, load
from neuralprophet import save, load, set_random_seed
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")
set_random_seed(0)

C:\Users\User\AppData\Local\Temp\ipykernel_2348\838273706.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel('C:/Users/User/Desktop/vscode/jinseok/train_total_var2.xlsx')
data

,ds,y,45_temp,45_max_Temp,45_min_Temp,45_hum,45_wind,45_rain,45_sun_Time,45_sun_Qy,45_condens_Time,45_gr_Temp,45_soil_Temp,45_soil_Wt
0,2015-02-14,0.0,0.11,12.5,-12.8,68.74,0.64,0.21,210.36,5.30,66.56,-0.08,1.24,13.62
1,2015-02-15,14570.0,0.27,12.5,-12.8,69.28,0.61,0.21,206.16,5.24,67.02,0.07,1.24,13.67
2,2015-02-16,7450.0,0.49,12.5,-12.8,70.06,0.59,0.42,200.30,5.15,82.34,0.27,1.24,13.88
3,2015-02-17,450.0,0.69,12.5,-11.6,70.35,0.60,0.42,197.76,5.12,94.08,0.46,1.27,14.15
4,2015-02-18,0.0,0.78,12.5,-11.6,70.21,0.62,0.42,193.04,5.07,94.08,0.56,1.31,14.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3238,2023-12-27,0.0,3.38,22.6,-11.4,66.68,0.52,1.24,210.66,4.87,122.31,3.59,7.97,16.10
3239,2023-12-28,0.0,3.34,22.6,-11.4,66.98,0.52,1.24,209.58,4.82,122.40,3.53,7.77,16.08
3240,2023-12-29,0.0,3.26,22.6,-11.4,67.09,0.51,1.24,208.38,4.77,122.89,3.44,7.59,16.06
3241,2023-12-30,0.0,3.16,22.6,-11.4,67.31,0.51,1.25,203.63,4.69,125.01,3.33,7.42,16.06


In [3]:
from neuralprophet import set_log_level
set_log_level("ERROR")
set_random_seed(0)
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Ignore specific warning by category
warnings.filterwarnings("ignore", category=DeprecationWarning)

    #Dependence Variables
col_lst = ['45_temp', '45_max_Temp', '45_min_Temp', '45_hum', '45_wind', '45_rain', '45_sun_Time', '45_sun_Qy', '45_condens_Time', '45_gr_Temp', '45_soil_Temp', '45_soil_Wt']

# configure_trainer(early_stopping=False,learning_rate=0.02,epochs=100)

m = NeuralProphet(
            n_forecasts=30*12,#일뒤에 regressor 지연 반영
            weekly_seasonality=1, #주간 계절성 설정
            n_lags=7, #일간 계절성 설정
            # d_hidden=128, #은닉층 뉴런 설정
            learning_rate=0.02, #학습률 설정
            # batch_size=30, 
            # epochs=100, #학습 횟수
            )
m = m.add_lagged_regressor(names=col_lst)

In [4]:
m.fit(data, freq="D")


Epoch 80: 100%|██████████| 80/80 [00:00<00:00, 141.28it/s, loss=0.0321, v_num=26, MAE=2.57e+4, RMSE=4.21e+4, Loss=0.032, RegLoss=0.000]   


,MAE,RMSE,Loss,RegLoss,epoch
0,446548.218750,567118.687500,1.320537,0.0,0
1,313063.468750,416011.718750,0.896355,0.0,1
2,206275.171875,291805.312500,0.559095,0.0,2
3,125786.476562,190966.593750,0.309888,0.0,3
4,75581.210938,119393.609375,0.160720,0.0,4
...,...,...,...,...,...
75,25732.902344,42103.515625,0.032027,0.0,75
76,25728.781250,42087.203125,0.032012,0.0,76
77,25705.589844,42065.746094,0.031972,0.0,77
78,25700.710938,42056.652344,0.031963,0.0,78


In [5]:
import warnings
warnings.filterwarnings('ignore')

future_np = m.make_future_dataframe(data, periods=365, n_historic_predictions=len(data))
 
# st.write(future)

forecast_future = m.predict(future_np)
forecast_value = m.get_latest_forecast(forecast_future, include_history_data=False)
forecast_value.fillna(0,inplace=True)

forecast_value['origin-0'] = forecast_value['origin-0'].apply(np.int64)
forecast_value['origin-0'] [forecast_value['origin-0'] <0]=0

forecast_value= forecast_value[['ds','origin-0']].rename(columns={'ds':'날짜','origin-0':'예측경매물량_Neural Prophet'})

Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 16.94it/s]


In [ ]:
# ## plot line graph using plotly

# import plotly.express as px

# fig = px.line(forecast_value, x='날짜', y='예측경매물량_Neural Prophet', title='예측경매물량_Neural Prophet')
# fig.show()

In [ ]:
forecast_value.to_excel('C:/Users/User/Desktop/vscode/jinseok/21.prediction_value_total_24year_45window_12x.xlsx', index=False)